#Полезные функции

In [ ]:
!pip install rdkit datamol molfeat xgboost catboost joblib

In [ ]:
#!pip install -U lightautoml

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import FunctionTransformer

import matplotlib.pyplot as plt
import seaborn as sns
import joblib

def mol_dsc_calc(mols):
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in mols.values)

# список конституционных и физико-химических дескрипторов из библиотеки RDKit
descriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
               "NHOHCount": Descriptors.NHOHCount,
               "NOCount": Descriptors.NOCount,
               "NumHAcceptors": Descriptors.NumHAcceptors,
               "NumHDonors": Descriptors.NumHDonors,
               "NumHeteroatoms": Descriptors.NumHeteroatoms,
               "NumRotatableBonds": Descriptors.NumRotatableBonds,
               "NumValenceElectrons": Descriptors.NumValenceElectrons,
               "NumAromaticRings": Descriptors.NumAromaticRings,
               "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
               "RingCount": Descriptors.RingCount,
               "MW": Descriptors.MolWt,
               "LogP": Descriptors.MolLogP,
               "MR": Descriptors.MolMR,
               "TPSA": Descriptors.TPSA,
               "Molecular Weight": Descriptors.MolWt}

def rdkit_fp(smiles_column: pd.Series, radius=3, nBits=2048, useChirality=False):
    # morganFP_rdkit
    def desc_gen(mol):
        mol = Chem.MolFromSmiles(mol)
        bit_vec = np.zeros((1,), np.int16)
        DataStructs.ConvertToNumpyArray(
            AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits, useChirality=useChirality), bit_vec)
        return bit_vec

    return pd.DataFrame.from_records(smiles_column.apply(func=desc_gen), columns=[f'bit_id_{i}' for i in range(nBits)])


def rdkit_2d(smiles_column: pd.Series):
    # 2d_rdkit
    descriptors = {i[0]: i[1] for i in Descriptors._descList}
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in smiles_column)

In [ ]:
def extract_smiles(raw_data: pd.DataFrame, smiles: pd.Series, add_bit_vec: bool=True, add_2d_rdkit: bool=False) -> pd.DataFrame:

    data = raw_data.copy()
    columns = data.columns

    descriptors_transformer = FunctionTransformer(mol_dsc_calc)
    X = descriptors_transformer.transform(smiles)
    data = data.join(X)

    if add_bit_vec:
        Y = rdkit_fp(smiles)
        data = data.join(Y)

    if add_2d_rdkit:
        Z = rdkit_2d(smiles)
        data = data.join(Z)

    return data

In [ ]:
def draw_hist(data, features, width=5):
  figure, axes = plt.subplots(ncols=2, nrows=len(features), figsize=(width, width*len(features)))
  for i, name in enumerate(features):
      axes[i, 0].set_title(name)
      sns.histplot(data[name], ax=axes[i, 0])
      axes[i, 1].set_title(name)
      sns.scatterplot(data[name], ax=axes[i, 1])

In [ ]:
def cut_quantiles(raw_data: pd.DataFrame, cols_to_cut: list=None, q_min: float=0.25, q_max: float=0.75) -> pd.DataFrame:

    data = raw_data.copy()

    quant1 = data[cols_to_cut].quantile(q_min)
    quant2 = data[cols_to_cut].quantile(q_max)
    quants = pd.concat([quant1, quant2], axis=1)

    for name in quants.index:
        data = data[quants.loc[name, q_min] <= data[name]]
        data = data[data[name] <= quants.loc[name, q_max]]

    return data

In [ ]:
def encode(data):
    import pandas as pd
    from sklearn.preprocessing import OneHotEncoder


    cat_cols = ["Strain", "Cell"]
    encoder = OneHotEncoder()

    # Fit and transform the categorical features
    encoded_data = encoder.fit_transform(data[cat_cols]).toarray()

    # Create a DataFrame with the one-hot encoded columns
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())

    # Concatenate the original dataset and the encoded dataset
    df_encoded = pd.concat([data, encoded_df], axis=1)

    # Drop original categorical columns
    df_encoded.drop(cat_cols, axis=1, inplace=True)
    joblib.dump(encoder, 'onehot_encoder_cc50.joblib')
    return df_encoded

In [ ]:
def scale(data):
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    scaler = MinMaxScaler()
    target_scaler = StandardScaler()
    cols = ["MW",	"LogP",	"MR",	"TPSA",	"Molecular Weight"]
    data_buf = data.copy()
    scaler.fit(data_buf[cols])
    y = pd.DataFrame(target_scaler.fit_transform(pd.DataFrame(data_buf["Standard Value"])), columns=["Standard Value"])
    X = pd.DataFrame(scaler.transform(data_buf[cols]), columns=cols)
    data_buf[cols] = X
    data_buf["Standard Value"] = y
    joblib.dump(scaler, 'scaler_cc50.joblib')
    joblib.dump(target_scaler, 'target_scaler_cc50.joblib')
    return data

#Подготовка датасета

In [ ]:
data = pd.read_csv("/content/ic50_df1 (1).csv")
smiles = data['Smiles']
data.drop(columns=["Smiles", "DOI"], inplace=True)
df_encoded = encode(data)
# data = data[data["Standard Value"] <= 0.3]
data_extract = extract_smiles(df_encoded, smiles)
data_extract.to_csv("ic50_extracted.csv", index=False)

In [ ]:
data_quant = cut_quantiles(data_extract, cols_to_cut=["MW",	"LogP",	"MR",	"TPSA",	"Molecular Weight"], q_min=0.03, q_max=0.97) #Мажквантильный размах
data_scale = scale(data_quant) # Стандартизация

In [ ]:
sns.histplot(data_extract["Standard Value"].iloc[:500])

#Обучение моделей

## CatBoost и XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
y = data_scale["Standard Value"]
X = data_scale.drop(columns=["Standard Value"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [ ]:
from xgboost import XGBRegressor
eval_set = [(X_train, y_train), (X_val, y_val)]
reg_xgboost = XGBRegressor(n_estimators=1500, max_depth=50, learning_rate=0.05, early_stopping_rounds=100)
reg_xgboost.fit(X_train, y_train, eval_set=eval_set, verbose=False)
print(reg_xgboost.score(X_test, y_test))
joblib.dump(reg_xgboost, 'reg_ic50_xgboost_1.joblib')

In [ ]:
from catboost import CatBoostRegressor

reg_catboost = CatBoostRegressor(iterations=1000, learning_rate=0.02, early_stopping_rounds=5)
reg_catboost.fit(X_train, y_train, logging_level="Silent", eval_set=(X_val, y_val), plot=True, plot_file="graph.txt")
print(reg_catboost.score(X_test, y_test))
joblib.dump(reg_catboost, 'reg_cc50_catboost_1.joblib')

##LightAutoML

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import torch

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

cols = list(data_scale.columns)

X = data_scale[cols]
df_train, df_test = train_test_split(X, random_state=42)

y_true = df_test["Standard Value"]
df_test.drop(columns=["Standard Value"])
automl = TabularAutoML(
    task = Task(
        name = 'reg',
        metric = r2_score),
    timeout=1000
)
oof_pred = automl.fit_predict(
    df_train,
    roles = {'target': "Standard Value"}
)
torch.save(automl, "model.pt")
test_pred = automl.predict(df_test)

In [ ]:
import joblib
joblib.dump(automl, 'automl_cc50.joblib')

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error

# test_pred_1 = test_pred.data
# y_true_1 = y_true.to_numpy()
print("r2_score: ", r2_score(y_true_1, test_pred_1))
print("mean_absolute_error:", mean_absolute_error(y_true_1, test_pred_1))
print("mean_absolute_percentage_error:", mean_absolute_percentage_error(y_true_1, test_pred_1))